In [17]:
## for data
import pandas as pd
import numpy as np
import re
from datetime import datetime
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for machine learning
from sklearn import metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
df_movies_full = pd.read_csv("datasets/to_use/movies_web_app_with_id_final.csv", dtype = {'movieId': int}) #still have UI information 
df_movies = df_movies_full.iloc[:, 7:]
df_movies_original = df_movies.copy() #still have movieId

In [19]:
df_movies_full

,title,imdb_rating,num_of_rating,title_only,year,poster_url,plot,movieId,Action,Adult,...,certificate_nan,num_rating_super_low,num_rating_low,num_rating_medium,num_rating_high,num_rating_super_high,imdb_rating_low,imdb_rating_medium,imdb_rating_high,imdb_rating_super_high
0,Toy Story (1995),8.3,997000.0,Toy Story,1995,https://m.media-amazon.com/images/M/MV5BMDU2ZW...,A cowboy doll is profoundly threatened and jea...,1,0,0,...,0,0,0,0,0,1,0,0,0,1
1,Jumanji (1995),7.0,351000.0,Jumanji,1995,https://m.media-amazon.com/images/M/MV5BZTk2Zm...,When two kids find and play a magical board ga...,2,0,0,...,0,0,0,0,0,1,0,0,1,0
2,Waiting to Exhale (1995),5.9,11000.0,Waiting to Exhale,1995,NaN,NaN,4,0,0,...,0,0,0,0,1,0,0,0,1,0
3,Tom and Huck (1995),5.5,11000.0,Tom and Huck,1995,NaN,NaN,8,0,0,...,0,0,0,0,1,0,0,0,1,0
4,Nixon (1995),7.1,32000.0,Nixon,1995,NaN,NaN,14,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82665,Üç Sevgili (1972),5.5,187.0,Üç Sevgili,1972,NaN,NaN,269115,0,0,...,1,1,0,0,0,0,0,0,1,0
82666,Üç arkadas (1972),6.9,494.0,Üç arkadas,1972,NaN,NaN,269116,0,0,...,1,1,0,0,0,0,0,0,1,0
82667,Üçüncü Sayfa (1999),7.3,3900.0,Üçüncü Sayfa,1999,NaN,NaN,269117,0,0,...,1,0,0,1,0,0,0,0,1,0
82668,Þrestir (2015),6.8,1600.0,Þrestir,2015,NaN,NaN,269118,0,0,...,1,0,0,1,0,0,0,0,1,0


In [20]:
df_movies_original

,movieId,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,certificate_nan,num_rating_super_low,num_rating_low,num_rating_medium,num_rating_high,num_rating_super_high,imdb_rating_low,imdb_rating_medium,imdb_rating_high,imdb_rating_super_high
0,1,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,2,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,4,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
3,8,0,0,1,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
4,14,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82665,269115,0,0,0,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,1,0
82666,269116,0,0,0,0,0,1,0,0,1,...,1,1,0,0,0,0,0,0,1,0
82667,269117,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,1,0
82668,269118,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,1,0


In [21]:
df_movies.set_index('movieId', inplace=True)

In [22]:
df_movies

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,certificate_nan,num_rating_super_low,num_rating_low,num_rating_medium,num_rating_high,num_rating_super_high,imdb_rating_low,imdb_rating_medium,imdb_rating_high,imdb_rating_super_high
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,1,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
8,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
14,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269115,0,0,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,1,0
269116,0,0,0,0,0,1,0,0,1,0,...,1,1,0,0,0,0,0,0,1,0
269117,0,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0


In [23]:
#dummy data
movie_id = int('55820')
movie_name = 'No Country for Old Men (2007)'


In [24]:
df_movie = pd.read_csv('datasets/to_use/movies_web_app_with_id_final.csv')


In [25]:
target_movie_cosine_sim_index = df_movies_original.index[df_movies_original['movieId'] == movie_id].values[0] #get 0-based index in cosine similarity array
target_movie_cosine_sim_index

5691

In [26]:
# df_movies_original[df_movies_original['movieId'] == movie_id]

In [27]:
target_movie = df_movies.loc[[movie_id]]
target_movie

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,certificate_nan,num_rating_super_low,num_rating_low,num_rating_medium,num_rating_high,num_rating_super_high,imdb_rating_low,imdb_rating_medium,imdb_rating_high,imdb_rating_super_high
movieId,,,,,,,,,,,,,,,,,,,,,
55820,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [28]:
normalised_df_movies = df_movies.astype(np.float32)
# cosine_sim = cosine_similarity(normalised_df_movies)

cosine_sim = cosine_similarity(normalised_df_movies, target_movie)

In [29]:
cosine_sim

array([[0.33333333],
       [0.16666667],
       [0.33333333],
       ...,
       [0.20412415],
       [0.20412415],
       [0.        ]])

In [30]:
similar_movies = list(enumerate(cosine_sim.flatten()))
# similar_movies
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

sorted_similar_movies

[(202, 1.0000000000000002),
 (293, 1.0000000000000002),
 (569, 1.0000000000000002),
 (977, 1.0000000000000002),
 (1871, 1.0000000000000002),
 (2616, 1.0000000000000002),
 (5152, 1.0000000000000002),
 (5352, 1.0000000000000002),
 (5691, 1.0000000000000002),
 (9982, 1.0000000000000002),
 (33272, 1.0000000000000002),
 (47288, 1.0000000000000002),
 (6, 0.9128709291752769),
 (58, 0.9128709291752769),
 (143, 0.9128709291752769),
 (396, 0.9128709291752769),
 (517, 0.9128709291752769),
 (560, 0.9128709291752769),
 (1049, 0.9128709291752769),
 (1614, 0.9128709291752769),
 (2000, 0.9128709291752769),
 (3522, 0.9128709291752769),
 (4028, 0.9128709291752769),
 (4718, 0.9128709291752769),
 (4910, 0.9128709291752769),
 (7181, 0.9128709291752769),
 (23749, 0.9128709291752769),
 (32390, 0.9128709291752769),
 (41502, 0.9128709291752769),
 (65060, 0.9128709291752769),
 (73028, 0.9128709291752769),
 (73029, 0.9128709291752769),
 (23, 0.8333333333333336),
 (226, 0.8333333333333336),
 (274, 0.8333333333333

In [31]:
def get_title_from_index(index):
    movieId = df_movies_original.iloc[index]['movieId']
    return df_movies_full[df_movies_full.movieId == movieId]["title"].values[0]
target_movie_cosine_sim_index = df_movies_original.index[df_movies_original['movieId'] == movie_id].values[0] #get 0-based index in cosine similarity array of the target movie
i=0
final_dict = {}
for movie in sorted_similar_movies:
    if movie[0] == target_movie_cosine_sim_index:
        continue
#     print(get_title_from_index(movie[0]))
    title = get_title_from_index(movie[0])
    score = movie[1]
    final_dict[i] = {
        'title': title,
        'score': score
    }
    i=i+1
    if i>=50:
        break
        
for movie in final_dict:
    print(final_dict[movie])

{'title': "Carlito's Way (1993)", 'score': 1.0000000000000002}
{'title': 'The Silence of the Lambs (1991)', 'score': 1.0000000000000002}
{'title': "Miller's Crossing (1990)", 'score': 1.0000000000000002}
{'title': 'The Untouchables (1987)', 'score': 1.0000000000000002}
{'title': 'Traffic (2000)', 'score': 1.0000000000000002}
{'title': 'Road to Perdition (2002)', 'score': 1.0000000000000002}
{'title': 'Lucky Number Slevin (2006)', 'score': 1.0000000000000002}
{'title': 'The Departed (2006)', 'score': 1.0000000000000002}
{'title': 'Nightcrawler (2014)', 'score': 1.0000000000000002}
{'title': 'Crash (I) (2004)', 'score': 1.0000000000000002}
{'title': 'Joker (I) (2019)', 'score': 1.0000000000000002}
{'title': 'Casino (1995)', 'score': 0.9128709291752769}
{'title': 'Taxi Driver (1976)', 'score': 0.9128709291752769}
{'title': 'Pulp Fiction (1994)', 'score': 0.9128709291752769}
{'title': 'The Godfather (1972)', 'score': 0.9128709291752769}
{'title': 'Reservoir Dogs (1992)', 'score': 0.9128709

In [32]:
df_movies_full[df_movies_full.title_only == 'The League of Extraordinary Gentlemen']

,title,imdb_rating,num_of_rating,title_only,year,poster_url,plot,movieId,Action,Adult,...,certificate_nan,num_rating_super_low,num_rating_low,num_rating_medium,num_rating_high,num_rating_super_high,imdb_rating_low,imdb_rating_medium,imdb_rating_high,imdb_rating_super_high
73757,The League of Extraordinary Gentlemen (2003),5.8,178000.0,The League of Extraordinary Gentlemen,2003,https://m.media-amazon.com/images/M/MV5BZTFlOT...,"In an alternate Victorian Age world, a group o...",260207,1,0,...,0,0,0,0,0,1,0,0,1,0
